<a href="https://colab.research.google.com/github/andrjohns/ProbAI-2024/blob/main/BayesianWorkflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Workflow - Probabilistic AI School 2024

## Preparation


In [ ]:
# Use a repository of pre-built package binaries to speed-up installation
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
system("./add_cranapt_jammy.sh")

# Install the R Packages we'll be using
install.packages(c("dplyr", "bayesplot", "cmdstanr"),
                  repos = c("https://stan-dev.r-universe.dev", getOption("repos")))

# Install and setup CmdStan
download.file("https://github.com/stan-dev/cmdstan/releases/download/v2.35.0/colab-cmdstan-2.35.0.tgz",
              "cmdstan-2.35.0.tgz")
utils::untar("cmdstan-2.35.0.tgz")
cmdstanr::set_cmdstan_path("cmdstan-2.35.0")

In this tutorial we will be using the [cmdstanr](https://mc-stan.org/cmdstanr/articles/cmdstanr.html) R interface to CmdStan:

In [ ]:
# Download the files for the tutorial
system("git clone https://github.com/andrjohns/ProbAI-2024")
setwd("ProbAI-2024")

In addition to cmdstanr, we'll be using the [bayesplot](https://mc-stan.org/bayesplot/articles/graphical-ppcs.html) package for our graphical model checking, so let's load that library as well:

In [ ]:
library(cmdstanr)
library(bayesplot)
library(ggplot2)
library(dplyr)

## Epilepsy RCT Workflow
### Data
Let's load our dataset and look at the general structure:


In [ ]:
epilepsy_rct <- readRDS("epilepsy.rds")
head(epilepsy_rct)

### Initial Model

We've decided to use a normal linear regression as our initial attempt for modelling the data:

$$
y_i \sim \textbf{N}(\alpha + x_i^T\beta, \sigma) \\
\alpha \sim N(0,10) \\
\beta \sim N(0,5) \\
\sigma \sim \textbf{N}^+(0,5)
$$

Let's have a look at how we would specify this in Stan:

In [ ]:
cat(readLines("Stan/normal_glm.stan"), sep = "\n")

Now that we have our model, and we've defined the data we'll need, let's structure our epilepsy observations to the right format:

In [ ]:
epilepsy_prep <- epilepsy_rct |>
  mutate(Trt = as.numeric(Trt) - 1,
         visit = as.numeric(visit) - 1,
         treat_x_visit = Trt * visit)

epilepsy_stan <- list(
  N = length(unique(epilepsy_prep$patient)),
  T = length(unique(epilepsy_prep$visit)),
  K = 5,
  x = epilepsy_prep[,c("Trt", "visit", "treat_x_visit", "zAge", "zBase")],
  y = epilepsy_prep$count,
  sample_prior = 1
)

Now we're ready to fit our model! Remember that Stan is a *compiled* language, so first we need to compile our Stan model into an executable:

In [ ]:
mod <- cmdstan_model("Stan/normal_glm.stan")

Now that it's compiled, we can begin the simulation process:

In [ ]:
fit <- mod$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2024
)

To perform our prior-predictive checks, we can use the `ppd_dens_overlay` function from the `bayesplot` package to plot the densities of the simulated data:

In [ ]:
ppd_dens_overlay(fit$draws(variables = "ypred", format = "draws_matrix")[1:25, ])

### Model 2: Poisson GLM

We've decided to use a Poisson Generalised Linear Model with a log-link as our second attempt for modelling the data:

$$
y_i \sim Poisson(\lambda_i) \\
\lambda_i = \exp(\alpha + x_i^T\beta) \\
\alpha \sim N(0,5) \\
\beta_{1:4} \sim N(0,1)
$$

Let's have a look at how we would specify this in Stan:

In [ ]:
cat(readLines("Stan/poisson.stan"), sep = "\n")

And let's check our new prior-predictive results:

In [ ]:
mod <- cmdstan_model("Stan/poisson.stan")
fit <- mod$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2024
)
ppd_dens_overlay(fit$draws(variables = "ypred", format = "draws_matrix")[1:25, ])

Much better! Let's fit the model to the observed data now:

In [ ]:
epilepsy_stan$sample_prior <- 0
fit <- mod$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2024
)

Let's check our diagnostics to see how the sampling went. First up, do the traceplots indicate that the chains have converged?

In [ ]:
mcmc_trace(fit$draws("beta"))

Looking good! Let's also check our R-hat statistic and effective sample sizes:

In [ ]:
fit$summary("beta")

Not bad! Let's have a look at our posterior-predictive checks:

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$y,
                 fit$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

It looks like there are some areas where our model doesn't represent our data very well, let's see what we can do about that.

### Random-Effects Model

We'll add a random intercept for each individual, to relax the assumption of equal mean and variance in the Poisson:

$$
y_i \sim Poisson(\lambda_i\theta_i) \\
\lambda_i = \exp(\alpha + x_i^T\beta) \\
\alpha \sim N(0,5) \\
\beta_{1:4} \sim N(0,1) \\
\theta_i \sim Gamma(\phi,\phi) \\
\phi \sim Cauchy^+(0,5)
$$

How does this look in our Stan model?

In [ ]:
cat(readLines("Stan/poisson_gamma.stan"), sep = "\n")

We also need to add an indicator for each individual in our dataset:

In [ ]:
epilepsy_stan$ID <- as.numeric(epilepsy_prep$patient)

Now let's follow the same process of compiling our model and then running sampling:

In [ ]:
mod_ranef <- cmdstan_model("Stan/poisson_gamma.stan")
fit_ranef <- mod_ranef$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

How do our convergence diagnostics and effective sample sizes look?

In [ ]:
mcmc_trace(fit_ranef$draws("beta"))
fit_ranef$summary("beta")

How about our fit to the data?

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$y,
                 fit_ranef$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

Much better! Now that we have a possible model, let's look at making it a little more efficient

### Marginalisation

Remember that we can represent the Poisson with a Gamma-distributed random effect as a Negative-Binomial parameterised by its mean and dispersion:

$$
\int Poisson(y | \lambda\theta) \cdot Gamma(\theta | \phi, \phi) d\theta = NB(y|\lambda, \phi)
$$

But don't just take my word for it, let's verify this in R by comparing the numerically integrated Poisson-Gamma with the Negative-Binomial:

In [ ]:
lambda <- 2.65
y <- 4
phi <- 1.5

poisson_gamma_pdf <- function(theta, y, lambda, phi) {
  exp(dpois(y, lambda * theta, log = TRUE) + dgamma(theta, shape = phi, rate = phi, log = TRUE))
}

integrate(poisson_gamma_pdf, 0, Inf, y, lambda, phi)
dnbinom(y, mu = lambda, size = phi)

Brilliant! Let's put this into practice with Stan:

In [ ]:
cat(readLines("Stan/nb.stan"), sep = "\n")

In [ ]:
mod_nb <- cmdstan_model("Stan/nb.stan")
fit_nb <- mod_nb$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$y,
                 fit_nb$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

Looking good again! How are our sampling runtimes looking?

In [ ]:
fit_ranef$time()$total
fit_nb$time()$total

Well that's a pretty impressive improvement! How much better can we do if we use the optimised GLM distributions in Stan?

In [ ]:
cat(readLines("Stan/nb_glm.stan"), sep = "\n")

In [ ]:
mod_nb_glm <- cmdstan_model("Stan/nb_glm.stan")
fit_nb_glm <- mod_nb_glm$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$y,
                 fit_nb_glm$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

In [ ]:
fit_ranef$time()$total
fit_nb$time()$total
fit_nb_glm$time()$total

Now that's a nice (and scalable) improvement!

### Compare Models

Now that we've finished developing our models (for now), how do they differ? Why would we prefer one over the other? Let's look at our inferences for the treatment effect:

In [ ]:
fit_ranef$summary("beta[1]")
fit_nb_glm$summary("beta[1]")

The more efficient sampling of the NG-GLM model resulted in less (computational) uncertainty in our estimates, a narrower posterior and greater effective sample size.

We can also see this by plotting the treatment effect posterior for each model:

In [ ]:
mcmc_dens(fit_ranef$draws("beta[1]")) + coord_cartesian(xlim=c(-1,0.5))
mcmc_dens(fit_nb_glm$draws("beta[1]")) + coord_cartesian(xlim=c(-1,0.5))